#### Seasonal Vorticity Budget Terms

This notebook is for saving seasonally averaged vorticity budget terms (1958 - 2012 average) using JRA55 forced 1/4 deg global MOM6 simulation.

In [1]:
import xarray as xr
import numpy as np
from xgcm import Grid
from dask.diagnostics import ProgressBar
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import glob, os

from dask.distributed import Client
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = Client(cluster)

client

Client Scheduler: tcp://127.0.0.1:46422 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 406.01 GB


In [2]:
# Read Data

path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210308/"

ppdir = "OM4p25_JRA55do1.4_0netfw_cycle6_extradiags_v3/gfdl.ncrc4-intel18-prod/pp/"

save_file = "OM4p25_JRA55_Vorticity_Budget_Season.nc"

ds_grid = xr.open_dataset(path + ppdir + "ocean_monthly_extra/ocean_monthly_extra.static.nc")

ds = xr.open_mfdataset(path + ppdir + "ocean_monthly_extra/av/monthly_5yr/*.nc")

ds = ds.groupby('time.season').mean('time')

ds = xr.merge([ds, ds_grid])

In [3]:
print(ds)

<xarray.Dataset>
Dimensions:             (nv: 2, season: 4, xh: 1440, xq: 1440, yh: 1080, yq: 1080)
Coordinates:
  * xq                  (xq) float64 -299.6 -299.3 -299.1 ... 59.66 59.91 60.16
  * yh                  (yh) float64 -80.39 -80.31 -80.23 ... 89.73 89.84 89.95
  * nv                  (nv) float64 1.0 2.0
  * xh                  (xh) float64 -299.7 -299.5 -299.2 ... 59.53 59.78 60.03
  * yq                  (yq) float64 -80.35 -80.27 -80.19 ... 89.78 89.89 90.0
  * season              (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    hf_dudt_2d          (season, yh, xq) float64 dask.array<chunksize=(1, 1080, 1440), meta=np.ndarray>
    hf_dvdt_2d          (season, yq, xh) float64 dask.array<chunksize=(1, 1080, 1440), meta=np.ndarray>
    ubt_dt              (season, yh, xq) float64 dask.array<chunksize=(1, 1080, 1440), meta=np.ndarray>
    vbt_dt              (season, yq, xh) float64 dask.array<chunksize=(1, 1080, 1440), meta=np.ndarray>
    hf_PFu_2d           (se

In [4]:
# Create grid and interpolate depth, beta

OMEGA = 7.2921e-5
RAD_EARTH = 6.378e6

grid = Grid(ds, coords={'X': {'center': 'xh', 'right': 'xq'},
                        'Y': {'center': 'yh', 'right': 'yq'} }, periodic=[ ])

depth_u = grid.interp(ds['deptho'] * ds['areacello'], 'X',  boundary='fill')  / ds['areacello_cu']
depth_v = grid.interp(ds['deptho'] * ds['areacello'], 'Y',  boundary='fill') / ds['areacello_cv']
depth_q = grid.interp(depth_u * ds['areacello_cu'], 'Y',  boundary='fill') / ds['areacello_bu']

colh_u = grid.interp(ds['col_height'] * ds['areacello'], 'X',  boundary='fill') / ds['areacello_cu']
colh_v = grid.interp(ds['col_height'] * ds['areacello'], 'Y',  boundary='fill') / ds['areacello_cv']

beta_v = 2*OMEGA*np.cos(ds.geolat_v * np.pi /180.)/RAD_EARTH
beta_q = 2*OMEGA*np.cos(ds.geolat_c * np.pi /180.)/RAD_EARTH

In [5]:
# compute terms in vorticity budget

rho_0 = 1035.

BPT = xr.Dataset()

vmo_bv = (ds['vmo_2d'] / (rho_0 * ds['dxCv']))
vmo_bv = beta_q * grid.interp(vmo_bv, 'X',  boundary='fill')
BPT['vmo_bv'] = vmo_bv

umo = (ds['umo_2d'] / (rho_0 * ds['dyCu']))
umo = grid.interp(umo, 'Y',  boundary='fill')
BPT['umo'] = umo

BPT_1 = (( - grid.diff((ds['intz_PFu_2d'] + ds['intz_u_BT_accel_2d']) * ds.dxCu, 'Y', boundary='fill')
           + grid.diff((ds['intz_PFv_2d'] + ds['intz_v_BT_accel_2d']) * ds.dyCv, 'X', boundary='fill')) 
         / ds.areacello_bu)
BPT['BPT'] = BPT_1

BPT['depth'] = (depth_q.load())

Mass_Surf = (grid.interp(grid.interp(ds['wfo'] * ds['areacello'], 'X', boundary='fill'), 'Y',  boundary='fill') 
             * ds['Coriolis'] / (rho_0)) / ds['areacello_bu']
BPT['Qm'] = Mass_Surf

div_u = (grid.diff(ds['umo_2d'] / (rho_0), 'X', boundary='fill') + 
         grid.diff(ds['vmo_2d'] / (rho_0), 'Y', boundary='fill') ) / ds['areacello']
div_u = - (grid.interp(grid.interp(div_u * ds['areacello'], 'X', boundary='fill'), 'Y', boundary='fill')
        * ds['Coriolis']) / ds['areacello_bu']
BPT['div_u'] = div_u

BPT['fdhdt'] = (Mass_Surf - div_u)

Curl_dudt = ( - grid.diff(ds['hf_dudt_2d'] * colh_u * ds['dxCu'], 'Y', boundary='fill')
            + grid.diff(ds['hf_dvdt_2d'] * colh_v * ds['dyCv'], 'X', boundary='fill') ) / ds.areacello_bu
BPT['Curl_dudt'] = Curl_dudt

Curl_taus = ( - grid.diff((ds['taux'])* ds.dxCu, 'Y', boundary='fill')
                   + grid.diff((ds['tauy'])* ds.dyCv, 'X', boundary='fill') )/ ds.areacello_bu
Curl_taus = Curl_taus / (rho_0 )
BPT['Curl_taus'] = Curl_taus

Curl_taub = ( - grid.diff((-ds['taux_bot'])* ds.dxCu, 'Y', boundary='fill')
                   + grid.diff(-ds['tauy_bot'] * ds.dyCv, 'X', boundary='fill') )/ ds.areacello_bu
Curl_taub = Curl_taub / (rho_0 )
BPT['Curl_taub'] = Curl_taub

Curl_Hrv2 = ( - grid.diff((ds['intz_rvxv_2d'] + ds['intz_gKEu_2d']) * ds.dxCu, 'Y', boundary='fill')
           + grid.diff((ds['intz_rvxu_2d'] + ds['intz_gKEv_2d']) * ds.dyCv, 'X', boundary='fill') )/ ds.areacello_bu
BPT['Curl_NL'] = Curl_Hrv2

Curl_Hdiff2 = ( - grid.diff(ds['intz_diffu_2d'] * ds.dxCu, 'Y', boundary='fill')
             + grid.diff(ds['intz_diffv_2d'] * ds.dyCv, 'X', boundary='fill') )/ ds.areacello_bu
BPT['Curl_Hdiff'] = Curl_Hdiff2

Curl_Cor2 = ( - grid.diff((ds['intz_CAu_2d'] - ds['intz_gKEu_2d'] - ds['intz_rvxv_2d'])* ds.dxCu, 'Y', boundary='fill')
           + grid.diff((ds['intz_CAv_2d'] - ds['intz_gKEv_2d'] - ds['intz_rvxu_2d'])* ds.dyCv, 'X', boundary='fill'))/ ds.areacello_bu
BPT['Curl_Cor'] = Curl_Cor2

tmpx = (ds['hf_dudt_2d'] * colh_u - ds['intz_CAu_2d']-ds['intz_PFu_2d']-ds['intz_diffu_2d']-
       ds['intz_u_BT_accel_2d'] - ds['taux']/rho_0 + ds['taux_bot']/rho_0)
tmpy = (ds['hf_dvdt_2d'] * colh_v - ds['intz_CAv_2d']-ds['intz_PFv_2d']-ds['intz_diffv_2d']-
       ds['intz_v_BT_accel_2d'] - ds['tauy'] /rho_0 + ds['tauy_bot']/rho_0)
Curl_remap = ( - grid.diff(tmpx * ds.dxCu, 'Y', boundary='fill')
             + grid.diff(tmpy * ds.dyCv, 'X', boundary='fill') )/ ds.areacello_bu
BPT['Curl_remap'] = Curl_remap

In [6]:
ds_save = xr.Dataset()

ds_save['beta_V'] = BPT['vmo_bv']
ds_save['beta_V'].attrs['units'] = "m/s^2"
ds_save['beta_V'].attrs['standard_name'] = "Meridional Coriolis gradient x depth-integrated meridional velocity"

ds_save['BPT'] = BPT['BPT'] + BPT['Curl_Cor'] + BPT['vmo_bv'] + BPT['Qm'] - BPT['fdhdt']
ds_save['BPT'].attrs['units'] = "m/s^2"
ds_save['BPT'].attrs['standard_name'] = "Bottom Pressure Torque"

ds_save['Curl_Adv'] = (BPT['Curl_NL'] + BPT['Curl_remap'])
ds_save['Curl_Adv'].attrs['units'] = "m/s^2"
ds_save['Curl_Adv'].attrs['standard_name'] = "Curl of depth-integrated nonlinear advetion term"

ds_save['Curl_taus'] =  BPT['Curl_taus']
ds_save['Curl_taus'].attrs['units'] = "m/s^2"
ds_save['Curl_taus'].attrs['standard_name'] = "Curl of Surface Wind Stress / rho_0"

ds_save['Curl_taub'] =  BPT['Curl_taub']
ds_save['Curl_taub'].attrs['units'] = "m/s^2"
ds_save['Curl_taub'].attrs['standard_name'] = "- Curl of bottom boundary stress / rho_0"

ds_save['Curl_diff'] = BPT['Curl_Hdiff']
ds_save['Curl_diff'].attrs['units'] = "m/s^2"
ds_save['Curl_diff'].attrs['standard_name'] = "Curl of depth-integrated horizontal diffusion"

ds_save['Mass_flux'] = (- BPT['Qm'])
ds_save['Mass_flux'].attrs['units'] = "m/s^2"
ds_save['Mass_flux'].attrs['standard_name'] = " - Coriolis x Surface mass flux / rho_0"

ds_save['eta_dt'] = BPT['fdhdt']
ds_save['eta_dt'].attrs['units'] = "m/s^2"
ds_save['eta_dt'].attrs['standard_name'] = "Coriolis x d(eta)/dt"

ds_save['Curl_dudt'] = (-BPT['Curl_dudt'])
ds_save['Curl_dudt'].attrs['units'] = "m/s^2"
ds_save['Curl_dudt'].attrs['standard_name'] = " - Curl of depth-integrated du/dt"

ds_save = xr.merge([ds_save, ds_grid])

ds_save = ds_save.transpose('season','yq','yh','xq','xh')

In [7]:
print(ds_save)

<xarray.Dataset>
Dimensions:       (season: 4, xh: 1440, xq: 1440, yh: 1080, yq: 1080)
Coordinates:
  * xq            (xq) float64 -299.6 -299.3 -299.1 -298.9 ... 59.66 59.91 60.16
  * yq            (yq) float64 -80.35 -80.27 -80.19 -80.11 ... 89.78 89.89 90.0
  * season        (season) object 'DJF' 'JJA' 'MAM' 'SON'
  * xh            (xh) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * yh            (yh) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
Data variables:
    beta_V        (season, yq, xq) float64 dask.array<chunksize=(1, 1080, 1439), meta=np.ndarray>
    BPT           (season, yq, xq) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    Curl_Adv      (season, yq, xq) float64 dask.array<chunksize=(1, 1079, 1439), meta=np.ndarray>
    Curl_taus     (season, yq, xq) float64 dask.array<chunksize=(1, 1079, 1439), meta=np.ndarray>
    Curl_taub     (season, yq, xq) float64 dask.array<chunksize=(1, 1079, 1439), meta=np.ndarray>
    Curl_diff     (

In [8]:
# save data

save_file = "OM4p25_JRA55_Vorticity_Budget_Season.nc"
path1 = "/archive/Hemant.Khatri/MOM_Budget/OM4p25_JRA55/v3_FW_adjust_True/"

%time ds_save.load().to_netcdf(path1 + save_file)

CPU times: user 52.2 s, sys: 3.87 s, total: 56.1 s
Wall time: 1min 46s


In [9]:
# save data

save_file = "OM4p25_JRA55_Momentum_Budget_Season.nc"
path1 = "/archive/Hemant.Khatri/MOM_Budget/OM4p25_JRA55/v3_FW_adjust_True/"

%time ds.load().to_netcdf(path1 + save_file)

CPU times: user 1min 31s, sys: 10.4 s, total: 1min 41s
Wall time: 2min 58s


In [ ]:
client.close()
cluster.close()